In [2]:
# Importanción de librerías.
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests

In [3]:
# Credenciales de Spotify.
CLIENT_ID = 'a018711755a146b2a678a93741dc6041'
CLIENT_SECRET = '51fa44e4b57343f19c810b9a75089da6'

# Autenticación con Spotify.
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
# Búsqueda en la API de Spotify a través de la librería spotipy.
datos = sp.search(q="genre:rock year:2020", type='track', limit=50, offset=0)

In [5]:
# Géneros y rango de años elegidos.
genres = ["pop", "rock", "jazz", "hip hop"]
year_start = 2002
year_end = 2007

# Almacenar los resultados de canciones y álbumes en listas.
lista_canciones_albumes = [] 
lista_artistas = []

In [6]:
# Acceder a los detalles del álbum de la primera canción que aparece en la lista de resultados que te devuelve la API de Spotify.
datos['tracks']['items'][0]['album']['type']


'album'

In [27]:
# Bucle para recorrer los géneros de la lista 'genres'.
for genre in genres: 
    # Realizar la búsqueda en la API de Spotify, buscando canciones (tracks) y álbumes (albums).
    # Se busca por género y por rango de años, limitando los resultados a 50 y estableciendo un desplazamiento (offset) de 951.
    datos = sp.search(q=f"genre:{genre} year:{year_start}-{year_end}", type='track,album', limit=50, offset=901)

    # Recorremos la lista de canciones encontradas en los resultados de la búsqueda.
    for item in datos['tracks']['items']: # 'tracks' contiene las canciones encontradas.
        
        # Agregamos, para cada canción, un diccionario con detalles relevantes a la lista.
        lista_canciones_albumes.append(
            {
                "nombre_cancion":item['name'],
                "nombre_artista":item['artists'][0]['name'],
                'genre':genre,                 
                "type":  item['album']['type'],
                "year": item['album']['release_date'][:4],
                "id": item['id']
            }
        )

        # Agregamos el nombre del artista a la lista.
        lista_artistas.append(
            {
                "nombre_artista":item['artists'][0]['name'],
            }
        )

In [28]:
# Longitud de la lista de artistas.
len(lista_artistas)

3800

In [29]:
# Longitud de la lista de canciones_albumes.
len(lista_canciones_albumes)

3800

In [30]:
# Convierte la lista de diccionarios en un DataFrame de Pandas.
df_lista_artistas = pd.DataFrame(lista_artistas)

In [31]:
# Limpia la lista lista_artistas. Si un artista aparece más de una vez en la columna, ahora solo aparecerá una vez en el conjunto.
lista_artistas_limpia = set(df_lista_artistas["nombre_artista"])


In [32]:
# Convierte la lista de diccionarios en un DataFrame de Pandas.
df_lista_canciones_albumes = pd.DataFrame(lista_canciones_albumes)

# Guarda el DataFrame de canciones en un archivo CSV, eliminando el índice, para que los datos puedan ser utilizados más fácilmente fuera de Python.
df_lista_canciones_albumes.to_csv("canciones.csv", index= False)


In [34]:
# Convierte la lista de diccionarios en un DataFrame de Pandas.
df_artistas_limpia = pd.DataFrame(lista_artistas_limpia)

# Guarda el DataFrame de artistas en un archivo CSV, eliminando el índice, para que los datos puedan ser utilizados más fácilmente fuera de Python.
df_artistas_limpia.to_csv("artistas.csv", index = False)

In [35]:
# Como ya tenemos los dos archivos CSV con la información de artistas, canciones y álbumes, desde la API de last.fm sacamos la información de cada artista.

# Credenciales de last.fm.
API_KEY = 'bb92344bdb091e7c718014c1dcd0e28d'  
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'

In [36]:
# Declaración nueva lista para posteriormente añadir os datos
datos_artistas = []

In [37]:
# Bucle para recorrer la lista de artistas ya limpia.
for artista in lista_artistas_limpia:

    # Parámetros para hacer una solicitud a la API.
    params_info = {
        "method": "artist.getInfo", 
        "api_key": API_KEY,
        "format": "json", 
        "artist": artista
    }
    try:
        # Consulta de la API. Realizamos la solicitud GET a la API con los parámetros definidos.
        response_info = requests.get(BASE_URL, params = params_info)

        # Verificamos si la solicitud fue exitosa (código de estado 200), sino se imprime un mensaje de error con el nombre del artista y el código de estado.
        if response_info.status_code !=200:
            print(f"Error en la solicitud para {artista}: {response_info.status_code}")
            continue

        # Si la solicitud fue exitosa, convertimos la respuesta en formato JSON.
        data_json = response_info.json()

        # Extraemos la información del artista del JSON de la respuesta.
        artist_data = data_json.get('artist', {})

        # Extraemos el resumen biográfico del artista. Si no está disponible, se asigna "Información no disponible".
        resumen = artist_data.get('bio', {}).get('summary', "Información no disponible")

        # Extraemos la popularidad (número de oyentes). Si no está disponible, se asigna "No disponible".
        popularidad = artist_data.get('stats', {}).get('listeners', "No disponible")

        # Extraemos el número de reproducciones (playcount). Si no está disponible, se asigna "No disponible".
        reproducciones = artist_data.get('stats', {}).get('playcount', "No disponible")

        # Extraemos los artistas similares. Si no hay artistas similares, se asigna una lista vacía.
        similares = [sim['name'] for sim in artist_data.get('similar', {}).get('artist', [])]

        # Añadimos un diccionario con la información del artista a la lista declarada anteriormente.
        # El diccionario contiene el nombre del artista, el resumen biográfico, la popularidad, el número de reproducciones y los artistas similares.
        datos_artistas.append({"artista": artista, 
                               "resumen": resumen,
                                "popularidad": popularidad,
                                "reproducciones": reproducciones,
                                "similares": similares})
    
    # Si ocurre un error debido a que falta alguna clave en el JSON capturamos el KeyError y mostramos un mensaje indicando que falta información para ese artista.
    except KeyError:
        print(f"Falta información para el artista: {artista}")

    # Si ocurre cualquier otro error inesperado (por ejemplo, problemas de red o conexión con la API), capturamos la excepción general y mostramos el mensaje de error junto con el nombre del artista.
    except Exception as e:
        print(f"Ocurrió un error inesperado con {artista}: {e}")
        
    

In [38]:
# Convierte la lista de diccionarios en un DataFrame de Pandas.
df_datos_artistas = pd.DataFrame(datos_artistas)

# Guarda el DataFrame de datos_artistas en un archivo CSV, eliminando el índice, para que los datos puedan ser utilizados más fácilmente fuera de Python.
df_datos_artistas.to_csv("datos_artistas.csv", index = False)

In [ ]:
# Conectamos python con mysql para importar los datos a nuestra base de datos

In [1]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd


In [2]:
df = pd.read_csv('canciones.csv')
df

,nombre_cancion,nombre_artista,genre,type,year,id
0,Caminando por la vida,Melendi,pop,album,2005,3v1dCP3hk2djfWryqfp7sx
1,Por la boca vive el pez,Fito y Fitipaldis,pop,album,2006,24UStI5AYA7g5ufFDh3LH0
2,Soldadito marinero,Fito y Fitipaldis,pop,album,2004,0eVborSuxUeSg0meWYd9dZ
3,No Puedo Vivir Sin Ti,Los Ronaldos,pop,album,2007,1EC0JRoSi9YL5zExd3Euc3
4,Rosas,La Oreja de Van Gogh,pop,album,2003,4waqcUQWdj0yH26STWl2Rq
...,...,...,...,...,...,...
3795,Red White Blue,Abstrakt Intellekt,hip hop,album,2007,5SOZwDQqYj7Wjh6KbvbOCp
3796,Voice of a Generation,Abstrakt Intellekt,hip hop,album,2007,2GPL1rrLQ4N5spa6gBktj2
3797,Le Hip-Hop mon royaume,Rocca,hip hop,album,2002,5kp38Y1j7CEgqmYExn3syW
3798,Stupid (feat. Katey Red),Big Freedia,hip hop,album,2002,1Bt01zWszIppYc5PHJ0oeE


In [18]:
lista_canciones = df.values.tolist()
len(lista_canciones)

3800

In [22]:
lista_canciones

[['Caminando por la vida',
  'Melendi',
  'pop',
  'album',
  2005,
  '3v1dCP3hk2djfWryqfp7sx'],
 ['Por la boca vive el pez',
  'Fito y Fitipaldis',
  'pop',
  'album',
  2006,
  '24UStI5AYA7g5ufFDh3LH0'],
 ['Soldadito marinero',
  'Fito y Fitipaldis',
  'pop',
  'album',
  2004,
  '0eVborSuxUeSg0meWYd9dZ'],
 ['No Puedo Vivir Sin Ti',
  'Los Ronaldos',
  'pop',
  'album',
  2007,
  '1EC0JRoSi9YL5zExd3Euc3'],
 ['Rosas',
  'La Oreja de Van Gogh',
  'pop',
  'album',
  2003,
  '4waqcUQWdj0yH26STWl2Rq'],
 ['Princesas', 'Pereza', 'pop', 'album', 2005, '3hlWUNjAlbZ6nOjAbrivBv'],
 ['La casa por el tejado',
  'Fito y Fitipaldis',
  'pop',
  'album',
  2004,
  '428cFrawSqeLoyooQjAW9d'],
 ['Marta, Sebas, Guille y los demás',
  'Amaral',
  'pop',
  'album',
  2005,
  '30ddpolNHwXlrS5XO8MiiO'],
 ['Con la luna llena',
  'Melendi',
  'pop',
  'album',
  2006,
  '5VwYcCKNAgDqDJ9ah9tDGl'],
 ['Zapatillas',
  'El Canto Del Loco',
  'pop',
  'album',
  2005,
  '42VgiRyg0YjtRVLiPFNxPJ'],
 ['Dile', 'Don Om

In [23]:
## Extraer los IDs
ids = [item[-1] for item in lista_canciones]
## Comprobar duplicados
duplicates = set([id_ for id_ in ids if ids.count(id_) > 1])
# si hay duplicados, los añadimos a la lista duplicates, y la imprimimos
if duplicates:
    print(f"IDs duplicados encontrados: {duplicates}")
else:
    print("No hay IDs duplicados.")

IDs duplicados encontrados: {'1vgfNhZVIU37PVkLXSuek6', '45dGKmN6jw7PgK6Dw2rEp2', '2afMcfINXLx8zh6ul0jGqD', '3Z3QhZAZpqwZa1phsbQ3JZ', '1JCDSRbe6H1x5BWLgAKTsB', '46Y9Jp57brIX3pqfN7zd8W', '1YeGiYCrtG7Ztr77VsgnpK', '5dncXckT4fSDlyEL9QgzuY', '2H3kc8vwKWtgaDWs0Xw98Y', '1TtwCkFbGv89XmuUNmoINt', '3sScSMFoQybUQg1IMEfEsl', '7M9atfLnxgO5ybzMoXZMdq', '1daEm8rmKDc9Mxp7r6bwyM', '5tusuce6vJETlevXyuUQVn', '3xOHBpfwg3oaRvgi6xZXCw', '0Y5i43d7YtHWyYulOojDXP', '1OPUGXvJXEWZu6qqBr0VTF', '7MKO7rUZzjw8p87zdZDgqa', '5FiB1uNoGZE4PenzZd7Imu', '3dqgHG4Rksh3B04Dt3cVHu', '1GPaOD3LyQhDnjZq2wc3H4', '6XR1DdEDU6LMMj7Mlyuzyy', '2g2a5kDeZexbUTD8abcvm6', '5zpZUFF7YyPG90nkR88RYd', '6ENKfKUj6bRvhdXE2KRKMz', '1EHh1fC9FCKw31wotfXx7L', '6bxMLISz19aaGCUpZKcxh1', '03Ntkzzjkz7nFJldcPbL90', '1AM8QdDFZMq6SrrqUnuQ9P', '2quTytCTP9kKLvpveyO1mt', '0eVborSuxUeSg0meWYd9dZ', '1WBSFlSLyOZnSa8fj7a5yC', '7axRDn2DTKeNSjUhrhKmTv', '1lS060MuE8QTmDPFXhSCD3', '20dDpjJ59PUAIdbXMkIrmd', '58zHoX9U5VlSXLUUmWTfe2', '4C2zOFgrXxfNGNpYQWIcaH', '7AMuS6Uo

In [24]:
len(duplicates)

357

In [20]:
for cancion in lista_canciones:
    
lista_canciones_limpia = set(lista_canciones[])

TypeError: list indices must be integers or slices, not str

In [6]:
try:
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                 host='127.0.0.1',
                                database='music_stream')
    print("Conexión establecida con éxito")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)


Conexión establecida con éxito


In [15]:
my_cursor = cnx.cursor()

In [16]:
query_canciones = ("INSERT INTO AlbumesyCanciones (nombre_cancion, nombre_artista, genre, type, year, id) VALUES (%s,%s,%s,%s,%s,%s)")

In [17]:
my_cursor.executemany(query_canciones,lista_canciones)

IntegrityError: 1062 (23000): Duplicate entry '3v1dCP3hk2djfWryqfp7sx' for key 'albumesycanciones.PRIMARY'

In [12]:
cnx.commit

<bound method MySQLConnection.commit of <mysql.connector.connection.MySQLConnection object at 0x000001B8830448D0>>